In [59]:
import psycopg2
import dns.resolver as DNSResolver
import subprocess
import re
import time

In [18]:
connection = psycopg2.connect(user = "postgres", password = "kenneth1996", host = "127.0.0.1", port = "5432", database = "afrinic_db")
connection.autocommit = True
cursor = connection.cursor()
# Print PostgreSQL Connection properties
print ( connection.get_dsn_parameters(),"\n")

{'user': 'postgres', 'dbname': 'afrinic_db', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [19]:
def db_insert_func(data_list: list, tabname: str ,columns: list):
    try :
        data = data_list.__str__().replace('[','').replace(']','')
        cols = columns.__str__().replace('[','').replace(']','').replace("'","")
        sql_statement = """INSERT INTO {}({}) VALUES({})""".format(tabname, cols, data)
        cursor.execute(sql_statement)
        res = True
    except Exception as e:
        res = False
        print(e)
    return res

In [20]:
data_list = ['41.in-addr.arpa.', 'NS', 'ns1.afrinic.net.', 'v4']
tab = 'edns_reverse'
cols = ['reverse_ns', 'ns_type', 'nameserver','ip_type']
cols.__str__().replace('[','').replace(']','').replace("'","")

'reverse_ns, ns_type, nameserver, ip_type'

In [21]:
data_list = ['41.in-addr.arpa.', 'NS', 'ns1.afrinic.net.', 'v4']
tab = 'edns_reverse'
cols = ['reverse_ns', 'ns_type', 'nameserver','ip_type']
print(db_insert_func(data_list, tab, cols))

True


In [8]:
# Print PostgreSQL version
cursor.execute("SELECT * from edns_reverse;")
record = cursor.fetchone()
print("You are connected to - ", record)

ProgrammingError: syntax error at or near "DESC"
LINE 1: DESC edns_reverse
        ^


In [6]:
x = DNSResolver.query('google.com')

In [11]:
txt = "The rain in Spain"
x = re.search("^The.*Spain$", txt)

<DNS name google.com.>

In [22]:
edns_test_dict = {'dns_plain': ['dig', '+norec', '+noedns', 'soa']
    ,'edns_plain': ['dig', '+norec', '+edns=0', 'soa']
    ,'edns_unknw': ['dig', '+norec', '+edns=100', '+noednsneg', 'soa']
    ,'edns_unknwopt': ['dig', '+norec', '+ednsopt=100', 'soa']
    ,'edns_unknwflag': ['dig', '+norec', '+ednsflags=0x80', 'soa']
    ,'edns_dnssec': ['dig', '+norec', '+dnssec', 'soa']
    ,'edns_trunc': ['dig', '+norec', '+dnssec', '+bufsize=512', '+ignore', 'dnskey']
    ,'edns_unknwveropt': ['dig', '+norec', '+edns=100', '+noednsneg', '+ednsopt=100', 'soa']}

In [32]:
def run_dig_cmd(cmd: list):
    status = None
    edns_version = None
    result = subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8').split(';;')
    for line in result:
        if re.search('status:', line):
             status = line.split(',')[1].split(':')[1].strip()
        elif re.search('EDNS: version: 0', line):
            edns_version = 0
    return status, edns_version, result

In [66]:
def run_ednsComp_test(ns: str):
    zone = '.'.join(ns.split('.')[1:])
    # Reset results vars
    dns_plain, edns_plain, edns_unknw, edns_unknwopt, edns_unknwflag, edns_dnssec, edns_trunc, edns_unknwveropt = False, False, False, False, False, False, False, False
    # Test DNS plain resolution first
    dns_plain = True if run_dig_cmd(edns_test_dict['dns_plain'] + [zone, '@'+ns])[0] == 'NOERROR' else False
    if dns_plain:
        edns_plain = True if run_dig_cmd(edns_test_dict['edns_plain'] + [zone, '@'+ns])[0:2] == ('NOERROR', 0) else False
        if edns_plain:
            edns_unknw = True if run_dig_cmd(edns_test_dict['edns_unknw'] + [zone, '@'+ns])[0:2] == ('BADVERS', 0) else False
            edns_unknwopt = True if run_dig_cmd(edns_test_dict['edns_unknwopt'] + [zone, '@'+ns])[0:2] == ('NOERROR', 0) else False
            edns_unknwflag = True if run_dig_cmd(edns_test_dict['edns_unknwflag'] + [zone, '@'+ns])[0:2] == ('NOERROR', 0) else False
            edns_dnssec = True if run_dig_cmd(edns_test_dict['edns_dnssec'] + [zone, '@'+ns])[0:2] == ('NOERROR', 0) else False
            edns_trunc = True if run_dig_cmd(edns_test_dict['edns_trunc'] + [zone, '@'+ns])[0:2] == ('NOERROR', 0) else False
            edns_unknwveropt = True if run_dig_cmd(edns_test_dict['edns_unknwveropt'] + [zone, '@'+ns])[0:2] == ('BADVERS', 0) else False
    return (dns_plain, edns_plain, edns_unknw, edns_unknwopt, edns_unknwflag, edns_dnssec, edns_trunc, edns_unknwveropt)

In [67]:
stime = time.time()
print(run_ednsComp_test('dnsinfo1-cte-pt.3g.vodacom.co.za.'))
print(time.time() - stime)

(True, True, True, True, True, True, True, True)
5.849191665649414


In [51]:
print(subprocess.run(['dig', '+norec', '+edns=0', 'soa', '3g.vodacom.co.za.','dnsinfo1-cte-pt.3g.vodacom.co.za.'], stdout=subprocess.PIPE).stdout.decode('utf-8'))


; <<>> DiG 9.11.5-P4-5.1+b1-Debian <<>> +norec +edns=0 soa 3g.vodacom.co.za. dnsinfo1-cte-pt.3g.vodacom.co.za.
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 19182
;; flags: qr ra; QUERY: 1, ANSWER: 0, AUTHORITY: 5, ADDITIONAL: 6

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4000
;; QUESTION SECTION:
;3g.vodacom.co.za.		IN	SOA

;; AUTHORITY SECTION:
co.za.			77077	IN	NS	ns0.is.co.za.
co.za.			77077	IN	NS	ns2.dns.business.
co.za.			77077	IN	NS	ns.coza.net.za.
co.za.			77077	IN	NS	ns3.iafrica.com.
co.za.			77077	IN	NS	coza1.dnsnode.net.

;; ADDITIONAL SECTION:
ns0.is.co.za.		77077	IN	A	196.4.160.17
ns.coza.net.za.		85482	IN	A	206.223.136.200
ns.coza.net.za.		85482	IN	AAAA	2001:43f8:30::200
coza1.dnsnode.net.	85482	IN	A	194.146.106.74
coza1.dnsnode.net.	85482	IN	AAAA	2001:67c:1010:18::53

;; Query time: 4 msec
;; SERVER: 10.77.71.15#53(10.77.71.15)
;; WHEN: Tue Oct 08 11:25:19 WAT 2019
;; MSG SIZE  rcvd: 286

;; Got answer:
;; ->>HEAD

In [39]:
 test = run_dig_cmd('google.com')

In [41]:
print(test)

(['\n; <<>> DiG 9.11.5-P4-5.1+b1-Debian <<>> google.com\n', ' global options: +cmd\n', ' Got answer:\n', ' ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 49820\n', ' flags: qr rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1\n\n', ' OPT PSEUDOSECTION:\n; EDNS: version: 0, flags:; udp: 4000\n', ' QUESTION SECTION:\n;google.com.\t\t\tIN\tA\n\n', ' ANSWER SECTION:\ngoogle.com.\t\t300\tIN\tA\t216.58.198.174\n\n', ' Query time: 137 msec\n', ' SERVER: 10.77.71.15#53(10.77.71.15)\n', ' WHEN: Sun Oct 06 17:09:39 WAT 2019\n', ' MSG SIZE  rcvd: 55\n\n'], 'NOERROR')
